In [ ]:
import sys
import argparse
from queue import PriorityQueue 
import itertools #https://www.geeksforgeeks.org/python-itertools-count/
#https://stackoverflow.com/questions/9287919/can-i-get-an-item-from-a-priorityqueue-without-removing-it-yet
from collections import deque
import heapq
import copy

#------------------------------------------Cell Class-----------------------------------------
class Cell():
    def __init__(self,row=None,col=None,val=None): #אתחול
        self.row=row
        self.col=col
        self.val=val
        
    def __eq__(self, other): # פונקציית השוואה
        if isinstance(other, Cell):
            return self.row==other.row and self.col==other.col and self.val==other.val
        return False

#------------------------------------------State Class-----------------------------------------
class State():
    def __init__(self,cellMatrix=None,lastState=None,cost=0,hirostic=0): #אתחול
        self.cellMatrix=cellMatrix # המטריצה מיוצגת כמערך של תאים
        self.lastState=lastState #הצומת שממנה הגיע המצב
        self.cost=cost # עלות
        self.hirostic=hirostic #הירוסטיקה
            
    def __eq__(self, other): #פונקציית השוואה לפי התאים 
        if isinstance(other, State):
            return self.cellMatrix==other.cellMatrix
        return True
    
    def __gt__(self, other): #Compare cost/hirostic
        #https://stackoverflow.com/questions/57487170/is-it-possible-to-pass-a-comparator-to-a-priorityqueue-in-python
        if(self.hirostic==other.hirostic): # במצב של BFS ההירוסטיקות שוות לאפס
            return self.cost > other.cost # אז נחשב לפי העלות
        return self.hirostic > other.hirostic # אחרת לפי הירוסטיקה
    
    def calculateHirostica(self): #חישוב הירוסטיקה לפי מס'הערכים שאינם נמצאים במקומם
        error = 0
        goal_state = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16] #במצב של 3 על 3 מגיע עד הספרה 9
        for i in range(len(self.cellMatrix)):
            if self.cellMatrix[i].val != goal_state[i]:
                error += 1
        return error 
    
    def __hash__(self): #פונקציית האש 
        string_val=""
        for cell in self.cellMatrix:
            if(cell.val>=10): #עבור 10-16 מקבל  ערכים של a-g
                string_val+=str(num_to_char(cell.val))
            else: 
                string_val+=str(cell.val)
        return hash(string_val)
    
#------------------------------------------ פונקציות עזר -----------------------------------------
def goalState(n): # פונקציה המחזירה את המצב הסופי עבור שני המצבים (3על3 ו-4על4)
    count = 1
    cellMatrix=[]
    for i in range(n):
        for j in range(n):
            cellMatrix.append(Cell(i+1,j+1,count))
            count+=1
    goal=State(cellMatrix)
    return goal

def num_to_char(value): # פונקציה שעוזרת לתת ערך האש חד ערכי כאשר המטריצה 4 על 4 ויש מספרים דו ספרתיים
    if value==10:
        return 'a'
    if value==11:
        return 'b'
    if value==12:
        return 'c'
    if value==13:
        return 'd'
    if value==14:
        return 'e'
    if value==15:
        return 'f'
    if value==16:
        return 'g'
    
def printMatrix(mat): # פונקציה המדפיסה מטריצה
    if(len(mat)==9):
        n=3
    else:
        n=4    
    for i in range(len(mat)):
        print("%d " % (mat[i].val), end = " ")
        if((i+1)%n==0):
            print()
    print()
        
def printPath(state,initial_state): # פונקציה המדפיסה את המסלול
    if(state==initial_state):
        printMatrix(initial_state.cellMatrix)
        return state
    beforeMe=printPath(state.lastState,initial_state)
    printMatrix(state.cellMatrix)
    return state  

def changeCell(state,pervcell,newcell): #פונקציה עזר שעושה החלפה בין התאים ב- succssorfunction
    newstate=copy.deepcopy(state) # יוצרת מצב חדש עם ההחלפה
    for cell in newstate.cellMatrix:
        if(cell==pervcell):
            cell.val=newcell.val
        if(cell==newcell):
            cell.val=pervcell.val
    return newstate

def succssorFunction(state):
        possibleSteps=[] #מערך המצבים האפשריים 
        for i in range(len(state.cellMatrix)):
            cell=state.cellMatrix[i] # עובר על כל תא במטריצה ובודק את האופציות המותרות
            if(cell.col -1 !=0): #left
                possibleSteps.append(changeCell(state,state.cellMatrix[i],state.cellMatrix[i-1]))
            if(len(state.cellMatrix)==9):
                if(cell.row -1 !=0): #Up
                    possibleSteps.append(changeCell(state,state.cellMatrix[i],state.cellMatrix[i-3]))
                    
                if(cell.row +1 <= 3): #Down
                    possibleSteps.append(changeCell(state,state.cellMatrix[i],state.cellMatrix[i+3]))
                    
                if(cell.col +1 <= 3): #Right
                    possibleSteps.append(changeCell(state,state.cellMatrix[i],state.cellMatrix[i+1]))
            else:
                if(cell.row -1 !=0): #Up
                    possibleSteps.append(changeCell(state,state.cellMatrix[i],state.cellMatrix[i-4]))
                if(cell.row +1 <= 4): #Down
                    possibleSteps.append(changeCell(state,state.cellMatrix[i],state.cellMatrix[i+4]))
                if(cell.col +1 <= 4): #Right
                    possibleSteps.append(changeCell(state,state.cellMatrix[i],state.cellMatrix[i+1]))
            temp_list = []
            for i in possibleSteps: #מוריד כפילויות 
                if i not in temp_list:
                    temp_list.append(i)
        return temp_list
    
#------------------------------------------ אלגוריתמים -----------------------------------------
def BFS(initial_state,goalstate):
    history={} #שומר את הצמתים שביקרנו בהם
    visit_states=0 #מס' הצמתים שביקרנו
    q=PriorityQueue()
    q.put(initial_state)
    while(q.qsize()!=0):
        curstate=q.get()
        history[hash(curstate)]=True #מעדכן את ערך התא
        visit_states+=1 
        if(curstate==goalstate): # בודק אם הגענו למצב הסופי
            print("Algorithm: BFS \n")
            printPath(curstate,initial_state) #מדפיס את המסלול
            print("Cost : "+ str(state.cost));
            print("Visit Count : "+ str(visit_states));
            return
        possibleStates=succssorFunction(curstate) # בודק מצבים אפשריים
        for state in possibleStates:
            if(history.get(hash(state)) is None):
                state.lastState=curstate
                state.cost=curstate.cost+1
                q.put(state)
                
def A_Star(initial_state,goalstate):
    history={} # שומר את הצמתים שביקרנו בהם
    visit_states=0
    q=PriorityQueue()
    q.put(initial_state)
    while(q.qsize()!=0):
        curstate=q.get()
        history[hash(curstate)]=True #מעדכן את ערך התא
        visit_states+=1
        if(curstate==goalstate):
            print("Algorithm: A* \n")
            printPath(curstate,initial_state)
            print("Cost : "+ str(curstate.cost));
            print("Visit Count : "+ str(visit_states));
            return
        possibleStates=succssorFunction(curstate) 
        for state in possibleStates:
            if(history.get(hash(state)) is None):
                state.lastState=curstate
                state.cost=curstate.cost+1
                state.hirostic=state.calculateHirostica() #מעדכן את ההירוסטיקה של כל מצב לפני ההכנסה לתור
                q.put(state)        

def ID_DFS(initial_state,goalstate):
    visit_states=0
    for depth in itertools.count(): # לולאה מ-0 עד העצירה
        history={}
        q=PriorityQueue()
        q.put(initial_state)
        while(q.qsize()!=0):
            curstate=q.get()
            history[hash(curstate)]=True #מעדכן את ערך התא
            visit_states+=1
            if(curstate==goalstate): # בודק מצב סופי
                print("Algorithm: Iterative Deepening \n")
                printPath(curstate,initial_state)
                print("Depth : "+ str(depth));
                print("Cost : "+ str(state.cost));
                print("Visit Count : "+ str(visit_states));
                return
            if(curstate.cost < depth):
                possibleStates=succssorFunction(curstate)
                for state in possibleStates:
                    if(history.get(hash(state)) is None):
                        state.lastState=curstate
                        state.cost=curstate.cost+1
                        q.put(state)

#------------------------------------------ הרצה -----------------------------------------                       
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('filename')
    parser.add_argument('algo')
    args = parser.parse_args()
    cells=[];
    script = sys.argv[0]
    file_name = sys.argv[1] #מקבל את שם הקובץ
    algo = sys.argv[2] #מקבל את שם האלגוריתם

    with open(file_name, 'r') as f:
        matrix = [[int(num) for num in line.split(' ')] for line in f]
        
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            cells.append(Cell(i+1,j+1,matrix[i][j])) # בונה מערך של תאים
    start=State(cells) # יוצר מצב התחלתי
    finalstate=goalState(len(matrix)) #מוצא את המצב הסופי בהתאם לגודל המטריצה
    if(algo=='BFS' or algo=='bfs'):
        BFS(start,finalstate)
    if (algo=='A*' or algo =='astar'):
        A_Star(start,finalstate)
    if (algo == 'ID' or algo == 'id'):
        ID_DFS(start,finalstate)
        
if __name__ == '__main__':
    main()